# 0. Setting up the environment

In [1]:
import os

In [36]:
# Download and setup the path
# Model name - Model that is going to save the checkpoint
# Pretrained Model - Tensorflow 2 detection zoo
# TF record - To generate the record file for the training and model
# Label map name - the file that contain the labels

CUSTOM_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8' 
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [37]:
paths = {
    'WORKSPACE_PATH': 'workspace',
    'TENSORFLOW_MODEL_PATH':os.path.join('tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('workspace','annotations'),
    'IMAGE_PATH': os.path.join('workspace','images'),
    'IMAGE_DATASET':os.path.join('workspace','images', 'dataset'), 
    'IMAGE_TEST_PATH' : os.path.join('workspace','images','test'),
    'IMAGE_TRAIN_PATH' : os.path.join('workspace','images','train'),
    'MODEL_PATH': os.path.join('workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('workspace','models',CUSTOM_MODEL_NAME), 
    'TFLITE_PATH':os.path.join('workspace','export-model',CUSTOM_MODEL_NAME, 'tfliteexport')
 }

In [38]:
files = {
    'PIPELINE_CONFIG':os.path.join('workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# Create the path
for path in paths.values():
    if not os.path.exists(path):
        !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
!pip install wget
import wget

In [ ]:
# Clone tensorflow model gardern

if not os.path.exists(os.path.join(paths['TENSORFLOW_MODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['TENSORFLOW_MODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection
# Restart the kernal after this steps 

# Using anacoda 
!conda install -c anaconda protobuf -y

!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
    
# Install the setup.py file in tensorflow
!cd tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd tensorflow/models/research/slim && pip install -e . 

In [ ]:
# Using verison 2.5.0
!pip install tensorflow-gpu==2.5.0
!pip install tensorflow-text==2.5.0

!pip install typeguard

In [ ]:
# Ensure the the script can run and is OK

VERIFICATION_SCRIPT = os.path.join(paths['TENSORFLOW_MODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
# If object detection have no module is found, restart the kernal and run again
import object_detection
import tensorflow as tf
tf.__version__

In [ ]:
# Unzip the folder
wget.download(PRETRAINED_MODEL_URL)
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 2. Create Label Map

In [ ]:
# Insert the label here
labels = [{'name':'dry syringe', 'id':1}, {'name':'dry plunger', 'id':2},
          {'name':'wet syringe', 'id':3}, {'name':'wet plunger', 'id':4},
          {'name':'moist syringe', 'id':5}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records
* Create 2 folder "train" and "test" under workspace/images and place your images

In [ ]:
# 80% train, 20% test
# use Data_Preparation.ipynb to clean up. Instruction at README.md

!python {TF_RECORD_SCRIPT_NAME} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {TF_RECORD_SCRIPT_NAME} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# 4. Copy Model Config to Training Folder

In [ ]:
# !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

Update the config file accordingly

In [ ]:
# Update the config file at the workspace/models/<modelname>/pipeline.config

# num_classes - 2 (Depending on how many label you have)
# fine_tune_checkpoint - (Path to the pre-trained-model checkpoint)
# "workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0"

# batch_size - 16 (Under train_config)

# fine_tune_checkpoint_type - "detection"

# For train_input_reader (Training)
# label_map_path - "workspace/annotations/label_map.pbtxt"
# input_path - "workspace/annotations/train.record" (File is generated from the generate_tfrecord.py)

# For eval_input_reader (testing)
# label_map_path - "workspace/annotations/label_map.pbtxt"
# input_path - "workspace/annotations/test.record" (File is generated from the generate_tfrecord.py)

# 5. Train the model

In [39]:
TRAINING_SCRIPT = os.path.join(paths['TENSORFLOW_MODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [40]:
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [41]:
# Copy the command and run this into the CMD line, ensure that you have run the correct environment
# You are able to quit anytime
# Best optimal value would be 0.3 and below

print(command)

python tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=workspace\models\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8 --pipeline_config_path=workspace\models\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\pipeline.config


# 6. Tensorflow Board (Optional)

To evaluate your model reading or learning, you can start a tensorboard by activating your environment again

```conda activate tensorflow```

Change to the folder directory path

```cd C:/tensorflow_build```

Start the tensorboard and enter the link that is provided in the cmd 
(http://localhost:6006/) Port number might vary

```tensorboard --logdir=workspace\models\<folder name>```

In [ ]:
command = "tensorboard --logdir={}".format(paths['CHECKPOINT_PATH'])
print(command)

# 7. Export to TFLite

In [29]:
TFLITE_SCRIPT = os.path.join(paths['TENSORFLOW_MODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
command = "python {} \
--pipeline_config_path={} \
--trained_checkpoint_dir={} \
--output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [ ]:
print(command)

In [ ]:
# Folder can be found at the export-model/tflite folder
!{command}

In [30]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,640,640,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL)

In [ ]:
# command = "tflite_convert \
# --output_file={} \
# --saved_model_dir={} \
# --inference_type=QUANTIZED_UINT8 \
# --input_shape= 1,300,300,3 \
# --input_array = normalized_input_image_tensor\
# --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
# --mean_values=128 --std_dev_values=128 \
# --default_ranges_min=0 --default_ranges_max=255 \
# --allow_custom_ops".format(TFLITE_MODEL, FROZEN_TFLITE_PATH)

In [ ]:
print(command)

In [ ]:
!{command}

In [ ]:
# Generate Label
# Label can be found at Section 2. Create Label Map
TFLITE_LABEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'labelmap.txt')
# labels = [{'name':'dry syringe', 'id':1}, {'name':'dry plunger', 'id':2}]

with open(TFLITE_LABEL, 'w') as f:
    for label in labels:
        f.write(label['name'])
        f.write('\n')

print("Done")

# 8.Evaluate the model

In [42]:
# Evaluating the precision
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [43]:
print(command)

python tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=workspace\models\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8 --pipeline_config_path=workspace\models\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\pipeline.config --checkpoint_dir=workspace\models\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8


An eval folder will be created and can be further eval by entering this command "tensorboard --logdir=." at the eval folder


# 9.Evaluate the image

In [ ]:
# Evaluting the picture

import os
import re
import numpy as np
from PIL import Image
from PIL import ImageDraw
import matplotlib.pyplot as plt

from random import seed
from random import randint
seed(1)

test_image_paths = [os.path.join(paths["IMAGE_TEST_PATH"],f) for f in os.listdir(paths["IMAGE_TEST_PATH"]) if os.path.splitext(f)[1] == ".jpg"]

def read_label_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    ret = {}
    for row_number, content in enumerate(lines):
        pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
        if len(pair) == 2 and pair[0].strip().isdigit():
            ret[int(pair[0])] = pair[1].strip()
        else:
            ret[row_number] = content.strip()
    return ret

def run_inference(interpreter, image):
    image = (np.float32(image) - input_mean) / input_std

    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    
    boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores = interpreter.get_tensor(output_details[2]['index'])[0]
    # num_detections = interpreter.get_tensor(output_details[3]['index'])[0]
    return boxes, classes, scores


In [ ]:

input_mean = 127.5
input_std = 127.5

# Creates tflite interpreter
interpreter = tf.lite.Interpreter(TFLITE_MODEL)
interpreter.allocate_tensors()
interpreter.invoke() # warmup

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

grayscale = True

for image_path in test_image_paths:
    print('Evaluating:', image_path)
    image = Image.open(image_path)
    
    if grayscale:
        image = image.convert('RGB') # Convert to RGB first
        
    
    image_width, image_height = image.size

    draw = ImageDraw.Draw(image)
        
    resized_image = image.resize((width, height))
    np_image = np.asarray(resized_image)
    input_tensor = np.expand_dims(np_image, axis=0)

    # Run inference
    boxes, classes, scores = run_inference(interpreter, input_tensor)


    # Draw results on image    
    labels = read_label_file(os.path.join(FROZEN_TFLITE_PATH, "labelmap.txt"))

    # Generate the color accordingly the number of our label
    colors = {}
    for i in range(len(labels)):
        value1 = randint(0, 255)
        value2 = randint(0, 255)
        value3 = randint(0, 255)
        colors[i] = (value1, value2, value3)


    %matplotlib inline
    for i in range(len(boxes)):
        if scores[i] > 0.5:
          
            try:
                ymin = int(max(1, (boxes[i][0] * image_height)))
                xmin = int(max(1, (boxes[i][1] * image_width)))
                ymax = int(min(image_height, (boxes[i][2] * image_height)))
                xmax = int(min(image_width, (boxes[i][3] * image_width)))

                draw.rectangle((xmin, ymin, xmax, ymax), outline=colors[int(classes[i])])
                draw.rectangle((xmin, ymin, xmax, ymin-10), fill=colors[int(classes[i])])

                text = labels[int(classes[i])] + ' ' + str(scores[i]*100) + '%'
                draw.text((xmin+2, ymin-10), text, fill=(0,0,0), width=2)
                print("Score for {}".format(text) )

            except:
                print("Can't draw box")
            finally:
                 display(image)

# 9. Export as TFLite TPU (Optional)

*Refer to README.md*

Reference
https://gist.github.com/NobuoTsukamoto/f48df315be490dcf0c76375c2e04ddb1#file-export_tfv2_lite_models-ipynb

Use "ssd_mobilenet_v2_320x320_coco17_tpu-8" only.

The size input of 300 x 300

In [ ]:
QUANTIZE_TPU_PATH = os.path.join(FROZEN_TFLITE_PATH, 'detect_quantize_edge.tflite')
QUANTIZE_TFLITE_PATH = os.path.join(FROZEN_TFLITE_PATH, 'detect_quantize.tflite')

In [ ]:
def convert_quant_full_int_model(input_path, output_path, width, height):
    input_width = width
    input_height = height
    
    def representative_data_gen():
        total_image = len([f for f in os.listdir(paths["IMAGE_TEST_PATH"]) if os.path.splitext(f)[1] == ".jpg"])
        dataset_list = tf.data.Dataset.list_files(paths["IMAGE_TEST_PATH"] + '\\*.jpg')
        
        for data in range(total_image):
            image = next(iter(dataset_list))
            image = tf.io.read_file(image)
            image = tf.io.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, (input_height, input_width))
#             image = image[np.newaxis,:,:,:]
#             image = image - 127.5
#             image = image * 0.007843
            image = tf.cast(image / 255., tf.float32)
            image = tf.expand_dims(image, 0)
            
            yield [image]

    
    print("Loading model...")
    converter = tf.lite.TFLiteConverter.from_saved_model(input_path, signature_keys=['serving_default'])
    print("Loading done...")
    
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    
    print("Gathering representive data...")
    converter.representative_dataset = representative_data_gen
    
    print("Converting...")
    tflite_full_integer_quant_model = converter.convert()
    print("Converting Done")
    
    open(output_path, "wb").write(tflite_full_integer_quant_model)

In [ ]:
convert_quant_full_int_model(FROZEN_TFLITE_PATH, QUANTIZE_TPU_PATH, 300,300)